# Problématique

SNCF-Transilien, l'opérateur des trains de banlieue d'Île-de-France, cherche à mieux anticiper l'augmentation du nombre de validations de cartes à puce par jour et par gare. Cette anticipation est cruciale pour améliorer la performance opérationnelle et proposer des services plus adaptés. Entre 2015 et 2019, le nombre de validations a augmenté d'environ 6% par an. Le but de ce challenge est de prédire le nombre de validations par jour et par gare à moyen et long terme.

## Objectif


L'objectif principal est de développer un modèle de machine learning capable de prédire le nombre de validations quotidiennes pour chaque gare du réseau SNCF-Transilien sur la période du 1er janvier 2023 au 30 juin 2023.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_percentage_error

# Importer les fichiers
train_data = pd.read_csv('data/train_f_x.csv')
test_data = pd.read_csv('data/test_f_x_THurtzP.csv')
y_train = pd.read_csv('data/y_train_sncf.csv')
y_example = pd.read_csv('data/y_exemple_sncf_d9so9pm.csv')

# Conversion de la colonne 'date' en type datetime
train_data['date'] = pd.to_datetime(train_data['date'])
test_data['date'] = pd.to_datetime(test_data['date'])

# Extraire les informations de 'index' dans y_train
y_train['date'] = y_train['index'].str.split('_').str[0]
y_train['station'] = y_train['index'].str.split('_').str[1]

# Conversion de la colonne 'date' en type datetime pour y_train
y_train['date'] = pd.to_datetime(y_train['date'])

# Fusionner les données d'entraînement avec les valeurs cibles
train_full = train_data.merge(y_train, on=['date', 'station'])

# Prendre un échantillon de 10% des données
train_sample = train_full.sample(frac=0.1, random_state=42)

# Séparer les features et la cible
X = train_sample.drop(columns=['y', 'index'])
y = train_sample['y']

# Diviser les données en ensembles d'entraînement et de validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Extraire des informations de la date
for df in [X_train, X_val, test_data]:
    df['day'] = df['date'].dt.day
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['day_of_week'] = df['date'].dt.dayofweek
    df['week_of_year'] = df['date'].dt.isocalendar().week

# Supprimer la colonne 'date' après l'extraction des features
X_train = X_train.drop(columns=['date'])
X_val = X_val.drop(columns=['date'])
test_data = test_data.drop(columns=['date'])

# Vérification des statistiques descriptives des valeurs cibles
print(y.describe())

# Distribution des valeurs cibles
plt.figure(figsize=(10, 6))
sns.histplot(y, bins=50, kde=True)
plt.title('Distribution des valeurs cibles (y)')
plt.xlabel('Nombre de validations')
plt.ylabel('Fréquence')
plt.show()


In [ ]:
# Appliquer une transformation logarithmique aux valeurs cibles
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)

# Vérifier la distribution après transformation
plt.figure(figsize=(10, 6))
sns.histplot(y_train_log, bins=50, kde=True)
plt.title('Distribution des valeurs cibles (y) après transformation logarithmique')
plt.xlabel('Log Nombre de validations')
plt.ylabel('Fréquence')
plt.show()
